<a href="https://colab.research.google.com/github/jeffheaton/app_deep_learning/blob/main/t81_558_class_08_4_bayesian_hyperparameter_opt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# T81-558: Applications of Deep Neural Networks

**Module 8: Kaggle Data Sets**

- Instructor: [Jeff Heaton](https://sites.wustl.edu/jeffheaton/), McKelvey School of Engineering, [Washington University in St. Louis](https://engineering.wustl.edu/Programs/Pages/default.aspx)
- For more information visit the [class website](https://sites.wustl.edu/jeffheaton/t81-558/).


# Module 8 Material

* Part 8.1: Introduction to Kaggle [[Video]](https://www.youtube.com/watch?v=7Mk46fb0Ayg&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_08_1_kaggle_intro.ipynb)
* Part 8.2: Building Ensembles with Scikit-Learn and PyTorch [[Video]](https://www.youtube.com/watch?v=przbLRCRL24&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_08_2_pytorch_ensembles.ipynb)
* Part 8.3: How Should you Architect Your PyTorch Neural Network: Hyperparameters [[Video]](https://www.youtube.com/watch?v=YTL2BR4U2Ng&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_08_3_pytorch_hyperparameters.ipynb)
* **Part 8.4: Bayesian Hyperparameter Optimization for PyTorch** [[Video]](https://www.youtube.com/watch?v=1f4psgAcefU&list=PLjy4p-07OYzulelvJ5KVaT2pDlxivl_BN) [[Notebook]](t81_558_class_08_4_bayesian_hyperparameter_opt.ipynb)
* Part 8.5: Current Semester's Kaggle [[Video]] [[Notebook]](t81_558_class_08_5_kaggle_project.ipynb)

# Google CoLab Instructions

The following code ensures that Google CoLab is running the correct version of TensorFlow.


In [1]:
# Startup Google CoLab
try:
    import google.colab
    COLAB = True
    print("Note: using Google CoLab")
except:
    print("Note: not using Google CoLab")
    COLAB = False

# Nicely formatted time string


def hms_string(sec_elapsed):
    h = int(sec_elapsed / (60 * 60))
    m = int((sec_elapsed % (60 * 60)) / 60)
    s = sec_elapsed % 60
    return "{}:{:>02}:{:>05.2f}".format(h, m, s)


# Make use of a GPU or MPS (Apple) if one is available.  (see module 3.2)
import torch
device = (
    "mps"
    if getattr(torch, "has_mps", False)
    else "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Note: using Google CoLab
Using device: mps


# Part 8.4: Bayesian Hyperparameter Optimization for PyTorch

Bayesian Hyperparameter Optimization is a method of finding hyperparameters more efficiently than a grid search. Because each candidate set of hyperparameters requires a retraining of the neural network, it is best to keep the number of candidate sets to a minimum. Bayesian Hyperparameter Optimization achieves this by training a model to predict good candidate sets of hyperparameters. [[Cite:snoek2012practical]](https://arxiv.org/pdf/1206.2944.pdf)

- [bayesian-optimization](https://github.com/fmfn/BayesianOptimization)
- [hyperopt](https://github.com/hyperopt/hyperopt)
- [spearmint](https://github.com/JasperSnoek/spearmint)


In [2]:
from scipy.stats import zscore
import pandas as pd
import logging
import os
logging.disable(logging.WARNING)


# Read the data set
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv",
    na_values=['NA', '?'])

# Generate dummies for job
df = pd.concat(
    [df, pd.get_dummies(df['job'], prefix="job", dtype=int)], axis=1)
df.drop('job', axis=1, inplace=True)

# Generate dummies for area
df = pd.concat(
    [df, pd.get_dummies(df['area'], prefix="area", dtype=int)], axis=1)
df.drop('area', axis=1, inplace=True)

# Missing values for income
med = df['income'].median()
df['income'] = df['income'].fillna(med)

# Standardize ranges
df['income'] = zscore(df['income'])
df['aspect'] = zscore(df['aspect'])
df['save_rate'] = zscore(df['save_rate'])
df['age'] = zscore(df['age'])
df['subscriptions'] = zscore(df['subscriptions'])

# Convert to numpy - Classification
x_columns = df.columns.drop('product').drop('id')
x = df[x_columns].values
dummies = pd.get_dummies(df['product'])  # Classification
products = dummies.columns
y = dummies.values

Now that we've preprocessed the data, we can begin the hyperparameter optimization. We start by creating a function that generates the model based on just three parameters. Bayesian optimization works on a vector of numbers, not on a problematic notion like how many layers and neurons are on each layer. To represent this complex neuron structure as a vector, we use several numbers to describe this structure.

- **dropout** - The dropout percent for each layer.
- **neuronPct** - What percent of our fixed 5,000 maximum number of neurons do we wish to use? This parameter specifies the total count of neurons in the entire network.
- **neuronShrink** - Neural networks usually start with more neurons on the first hidden layer and then decrease this count for additional layers. This percent specifies how much to shrink subsequent layers based on the previous layer. We stop adding more layers once we run out of neurons (the count specified by neuronPct).

These three numbers define the structure of the neural network. The commends in the below code show exactly how the program constructs the network.


In [3]:
import pandas as pd
import numpy as np
import time
import statistics
from scipy.stats import zscore
from sklearn import metrics
from sklearn.model_selection import StratifiedShuffleSplit
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Convert data to PyTorch tensors
x_tensor = torch.FloatTensor(x).to(device)
y_tensor = torch.LongTensor(np.argmax(y, axis=1)).to(
    device)  # Convert one-hot to index


class NeuralNetwork(nn.Module):
    def __init__(self, input_dim, dropout, neuronPct, neuronShrink):
        super(NeuralNetwork, self).__init__()

        layers = []
        neuronCount = int(neuronPct * 5000)
        layer = 0

        prev_count = input_dim
        while neuronCount > 25 and layer < 10:
            layers.append(nn.Linear(prev_count, neuronCount))
            prev_count = neuronCount
            layers.append(nn.PReLU())
            layers.append(nn.Dropout(dropout))
            neuronCount = int(neuronCount * neuronShrink)
            layer += 1

        layers.append(nn.Linear(prev_count, y.shape[1]))
        layers.append(nn.Softmax(dim=1))
        self.model = nn.Sequential(*layers)

    def forward(self, x):
        return self.model(x)

We can test this code to see how it creates a neural network based on three such parameters.


In [4]:
# Create and print the model
model = NeuralNetwork(x.shape[1], 0.2, 0.1, 0.25).to(device)
print(model)

NeuralNetwork(
  (model): Sequential(
    (0): Linear(in_features=47, out_features=500, bias=True)
    (1): PReLU(num_parameters=1)
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=500, out_features=125, bias=True)
    (4): PReLU(num_parameters=1)
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=125, out_features=31, bias=True)
    (7): PReLU(num_parameters=1)
    (8): Dropout(p=0.2, inplace=False)
    (9): Linear(in_features=31, out_features=7, bias=True)
    (10): Softmax(dim=1)
  )
)


We will now create a function to evaluate the neural network using three such parameters. We use bootstrapping because one training run might have "bad luck" with the assigned random weights. We use this function to train and then evaluate the neural network.


In [5]:
# Evaluation function
SPLITS = 2
EPOCHS = 500
PATIENCE = 10

def evaluate_network(learning_rate=1e-3,dropout=0.2,
                        neuronPct=0.1, neuronShrink=0.25):
    
    boot = StratifiedShuffleSplit(n_splits=SPLITS, test_size=0.1)
    mean_benchmark = []
    epochs_needed = []

    for train, test in boot.split(x, np.argmax(y, axis=1)):
        x_train = x_tensor[train]
        y_train = y_tensor[train]
        x_test = x_tensor[test]
        y_test = y_tensor[test]

        model = NeuralNetwork(x.shape[1],  
                        dropout=dropout,
                        neuronPct=neuronPct,
                        neuronShrink=neuronShrink).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)

        dataset_train = TensorDataset(x_train, y_train)
        loader_train = DataLoader(dataset_train, batch_size=32, shuffle=True)

        best_loss = float('inf')
        patience_counter = 0

        for epoch in range(EPOCHS):
            model.train()
            for batch_x, batch_y in loader_train:
                optimizer.zero_grad()
                outputs = model(batch_x)
                loss = criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()

            model.eval()
            with torch.no_grad():
                outputs_test = model(x_test)
                val_loss = criterion(outputs_test, y_test).item()

            if val_loss < best_loss:
                best_loss = val_loss
                patience_counter = 0
            else:
                patience_counter += 1

            if patience_counter >= PATIENCE:
                epochs_needed.append(epoch)
                break

        # Evaluate
        with torch.no_grad():
            model.eval()
            # Move predictions to CPU for evaluation
            pred = model(x_test).cpu().numpy()
            y_compare = y_test.cpu().numpy()
            score = metrics.log_loss(y_compare, pred)
            mean_benchmark.append(score)

    return -statistics.mean(mean_benchmark)

print(evaluate_network(learning_rate=1e-3, 
                       dropout=0.2,
                      neuronPct=0.1, 
                      neuronShrink=0.25))

-4.381666020591725


You can try any combination of our three hyperparameters, plus the learning rate, to see how effective these four numbers are. Of course, our goal is not to manually choose different combinations of these four hyperparameters; we seek to automate.


In [6]:
print(evaluate_network(
    dropout=0.2,
    neuronPct=0.1, 
    neuronShrink=0.25))

-5.754028869647901


First, we must install the Bayesian optimization package if we are in Colab.


In [7]:
# HIDE OUTPUT
!pip install bayesian-optimization

We will now automate this process. We define the bounds for each of these four hyperparameters and begin the Bayesian optimization. Once the program finishes, the best combination of hyperparameters found is displayed. The **optimize** function accepts two parameters that will significantly impact how long the process takes to complete:

- **n_iter** - How many steps of Bayesian optimization that you want to perform. The more steps, the more likely you will find a reasonable maximum.
- **init_points**: How many steps of random exploration that you want to perform. Random exploration can help by diversifying the exploration space.


In [8]:
from bayes_opt import BayesianOptimization
import time

# Supress NaN warnings
import warnings
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Bounded region of parameter space
pbounds = {'dropout': (0.0, 0.499),
           'learning_rate': (0.0, 0.1),
           'neuronPct': (0.01, 1),
           'neuronShrink': (0.01, 1)
           }

optimizer = BayesianOptimization(
    f=evaluate_network,
    pbounds=pbounds,
    verbose=2,  # verbose = 1 prints only when a maximum
    # is observed, verbose = 0 is silent
    random_state=1,
)

start_time = time.time()
optimizer.maximize(init_points=10, n_iter=20,)
time_took = time.time() - start_time

print(f"Total runtime: {hms_string(time_took)}")
print(optimizer.max)

|   iter    |  target   |  dropout  | learni... | neuronPct | neuron... |
-------------------------------------------------------------------------
| 1         | -9.167    | 0.2081    | 0.07203   | 0.01011   | 0.3093    |
| 2         | -8.29     | 0.07323   | 0.009234  | 0.1944    | 0.3521    |
| 3         | -9.167    | 0.198     | 0.05388   | 0.425     | 0.6884    |
| 4         | -9.167    | 0.102     | 0.08781   | 0.03711   | 0.6738    |
| 5         | -10.04    | 0.2082    | 0.05587   | 0.149     | 0.2061    |
| 6         | -12.71    | 0.3996    | 0.09683   | 0.3203    | 0.6954    |
| 7         | -8.29     | 0.4373    | 0.08946   | 0.09419   | 0.04866   |
| 8         | -11.84    | 0.08475   | 0.08781   | 0.1074    | 0.4269    |
| 9         | -8.29     | 0.478     | 0.05332   | 0.695     | 0.3224    |
| 10        | -10.04    | 0.3426    | 0.08346   | 0.02811   | 0.7526    |
| 11        | -10.04    | 0.003852  | 0.01619   | 0.7123    | 0.7671    |
| 12        | -12.08    | 0.2208    | 

As you can see, the algorithm performed 30 total iterations. This total iteration count includes ten random and 20 optimization iterations.
